# Negative Weighted Events

In this notebook, we adapt the negative events-based measure from van den Broucke et al. from 2014 in the paper: "Determining Process Model Precision and Generalization with Weighted Artificial Negative Events" (doi: 10.1109/TKDE.2013.130). <br>
With respect to the actual executions of a process, the measure focuses on negative events, where a negative event represents information about activities that were prevented from taking place in the first place. Since they are rarely recorded in reality, we induce them into the log artificially. <br>
So, we basically induce all elements that weren't fired at this specific position in the event log. For simplicity, the authors assume that all other events, outside the current event itself, are inserted at each position in the trace. Afterwards, we check which of these events could be fired at the current position in the corresponding position or not. If an event can be fired, we increase a counter for allowed generalizations $AG$ by one, and if not, we increase a counter for allowed generalizations $DG$ by one. <br>
The final measure is calculated as follows: <br>
For event log $E$ and process model $M$,
$$ Generalization (L,M) = AG\,/\, (AG+DG).$$
One has to be aware, that this is not the complete logic of the paper, which is implemented in a further step by introducing a scoring mechanism. This is done due to the fact that we have to assume the completeness of an event log to induce negative events and this is normally not true in reality, as an event log only represents a subset. The scoring mechanism will allow us to loosen this assumption by only assuming the completeness property on a small window before the execution of the current activity.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
import pickle
import multiprocessing
import numpy as np
from tqdm import tqdm
from src.utils import dfs, filter_silent_transitions

In [3]:
def update_global_variables_with(group_df, filtered_preceding_events_full, filtered_preceding_events,
                            filtered_succeeding_activities_updated, events, silent_transitions,trace_list, suffix_lists,
                                          current_trace_index, AG, DG):
    # list for all the activities that are enabled, starting from all activities that do not have any preceding activity
    enabled = [key for key, value in filtered_preceding_events_full.items() if not value]
    # initialise a list of already executed activities in this trace
    trace = []
    # iterate through each case/process execution
    for index, row in group_df.iterrows():
        # Get the current negative events based on the current activity to be executed
        negative_activities = [x for x in events if x != row['event_activity']]
        # it may happen that an activity is not present in the model but nevertheless executed in the log
        if row['event_activity'] in enabled:
            # check which elements in the negative activity list are enabled outside of the current activity
            enabled.remove(row['event_activity'])
        # get all the negative events that can not be executed in the process model at the moment
        disallowed = [value for value in negative_activities if value not in enabled]
        # add activity that has been executed to trace
        trace.append(row['event_activity'])
        #only look at all other traces and the own trace after the current position
        modified_suffix_list = suffix_lists[:current_trace_index] + suffix_lists[current_trace_index+1:]
        #cut out the already played trace such that we start only at the position after the current event
        cutted_current_trace = trace_list[current_trace_index][len(trace):]
        #generate the list of suffixes for the reversed traces for more efficient comparison
        reversed_activities_current = cutted_current_trace[::-1]
        suffixes_current = [reversed_activities_current[i:] for i in range(len(reversed_activities_current))]
        #append the suffixes of the current trace without the position
        modified_suffix_list.append(suffixes_current)
        #initialise scoring dictionary to save the values
        activity_dict = {activity: [1] for activity in negative_activities}
        #for each negative event
        for activity in negative_activities:
            #set the window size first as the lenght of the trace - 1, because the current value has already been added to the trace
            window_size = len(trace)-1
            #initialise the matching_window
            matching_window = 0
            if window_size == 0:
                #if the matching window is zero, break the loop, because we cannot compute a score (division by zero)
                break
            #for each trace that is not the original trace
            for execution in modified_suffix_list:
                #for each reversed suffix in the trace
                for suffix in execution:
                        #set a counter for the moving window
                        l = 1
                        #while the counter is smaller than the window size or the remaining trace of the suffix and the values at the corresponding positions are the same
                        while (l < min(window_size,len(suffix)-1)) & (trace[len(trace)-l] == suffix[l-1]):
                            #increment the matching window by one position and the counter by one
                            matching_window = matching_window + 1
                            l = l + 1
                        #after the loop has been broken calculate the sccore and save it in the activity list
                        score = (window_size - matching_window)/window_size
                        activity_dict[activity].append(score)
        #initialise empty lists to store the minimum value of the lists for each activity
        min_values_enabled = []
        min_values_negative = []
        #loop over all events in the log
        for value in negative_activities:
            if value in activity_dict:
                #compute the minimum value for eacch of the events outside of the current event
                min_val = min(activity_dict[value])
                #seperate the values that are enabled and disabled at this position
                if value in enabled:
                    min_values_enabled.append(min_val)
                elif value in disallowed:
                    min_values_negative.append(min_val)
        # update the values of allowed and disallowed generalizations based on the paper logic (incremend by 1-weight))
        AG = AG + (len(min_values_enabled)-sum(min_values_enabled))
        DG = DG + (len(min_values_negative)-sum(min_values_negative))
        # may happen that activities in the log are not in the process model
        if row['event_activity'] in filtered_succeeding_activities_updated:
            # get all possible new enabled activities
            possible_enabled = filtered_succeeding_activities_updated[row['event_activity']]
            # check if each activity has more than one directly preceding state
            for i in range(len(possible_enabled)):
                # check if an event has two or more activities that need to be executed before the event can take place, if not add events to enabled
                if len(filtered_preceding_events[possible_enabled[i]]) < 2:
                    enabled.append(possible_enabled[i])
                # if all succeeding events equal all preceding events, we have a flower model and almost everything is enabled all the time
                elif filtered_preceding_events[possible_enabled[i]] == filtered_succeeding_activities_updated[
                    possible_enabled[i]]:
                    enabled.append(possible_enabled[i])
                else:
                    # if yes, check if all the needed activities have already been performed in this trace
                    if all(elem in trace for elem in filtered_preceding_events[possible_enabled[i]]):
                        enabled.append(possible_enabled[i])
        # extend the list with all elements that do not have any preceding activity and are therefore enabled anyways in our process model
        enabled.extend([key for key, value in filtered_preceding_events_full.items() if not value])
        # delete all duplicates from the enabled list
        enabled = list(set(enabled))
    #increate the current trace rat
    current_trace_index = current_trace_index + 1
    return AG, DG

# Define the function that will be executed in parallel
def process_group_with(args):
    """
    Function to process a group of the event log in parallel
    :param args: set of variables for the measure calculation (see original function)
    :return: updated values for AG and DG, type: int
    """
    group_key, df_group, filtered_preceding_events_full, filtered_preceding_events, \
    filtered_succeeding_activities_updated, events, silent_transitions,trace_list, suffix_lists,current_trace_index, AG, DG = args
    AG, DG = update_global_variables_with(df_group, filtered_preceding_events_full, filtered_preceding_events,
                                     filtered_succeeding_activities_updated, events, silent_transitions,trace_list, suffix_lists,
                                          current_trace_index,AG, DG)
    return AG, DG

In [4]:
def negative_events_with_weighting_parallel(ocel, ocpn):
    """
    Function to calculate the negative events measure with weighting based on the used places inside an object-centric petri-net.
    :param ocel: object-centric event log for which the measure should be calculated, type: ocel-log
    :param ocpn: corresponding object-centric petri-net, type: object-centric petri-net
    :return generalization: final value of the formula, type: float rounded to 4 digits
    """
    # since the process execution mappings have lists of length one,
    # we create another dictionary that only contains the the value inside the list to be able to derive the case
    mapping_dict = {key: ocel.process_execution_mappings[key][0] for key in ocel.process_execution_mappings}
    # we generate a new column in the class (log) that contains the process execution (case) number via the generated dictionary
    ocel.log.log['event_execution'] = ocel.log.log.index.map(mapping_dict)
    # generate a list of unique events in the event log
    events = np.unique(ocel.log.log.event_activity)
    # dictionary to store each activity as key and a list of its prior states/places as value
    targets = {}
    # dictionary to store each activity as key and a list of its following states/places as value
    sources = {}
    for arc in tqdm(ocpn.arcs, desc="Check the arcs"):
        # for each arc, check if our target is a valid transition
        if arc.target in ocpn.transitions:
            # load all the prior places of a valid transition into a dictionary, where the key is the transition and the value
            # a list of all directly prior places
            if arc.target.name in targets:
                targets[arc.target.name].append(arc.source.name)
            else:
                targets[arc.target.name] = [arc.source.name]
        if arc.source in ocpn.transitions:
            # load all the following places of a valid transition into a dictionary, where the key is the transition and the value
            # a list of all directly following places
            if arc.source.name in sources:
                sources[arc.source.name].append(arc.target.name)
            else:
                sources[arc.source.name] = [arc.target.name]
    # generate an empty dictionary to store the directly preceding transition of an activity
    preceding_activities = {}
    # use the key and value of targets and source to generate the dictionary
    for target_key, target_value in targets.items():
        preceding_activities[target_key] = []
        for source_key, source_value in sources.items():
            for element in target_value:
                if element in source_value:
                    preceding_activities[target_key].append(source_key)
                    break
    # generate an empty dictionary to store the directly succeeding transition of an activity
    succeeding_activities = {}
    for source_key, source_value in sources.items():
        succeeding_activities[source_key] = []
        for target_key, target_value in targets.items():
            for element in source_value:
                if element in target_value:
                    succeeding_activities[source_key].append(target_key)
                    break
    # store the name of all silent transitions in the log
    silent_transitions = [x.name for x in ocpn.transitions if x.silent]
    # replace the silent transitions in the succeeding activities dictionary by creating a new dictionary to store the modified values
    succeeding_activities_updated = {}
    # Iterate through the dictionary
    for key, values in succeeding_activities.items():
        # Create a list to store the modified values for this key
        new_values = []
        # Iterate through the values of each key
        for i in range(len(values)):
            # Check if the value is in the list of silent transitions
            if values[i] in silent_transitions:
                # Replace the value with the corresponding value from the dictionary
                new_values.extend(succeeding_activities[values[i]])
            else:
                # If the value is not in the list of silent transitions, add it to the new list
                new_values.append(values[i])
        # Add the modified values to the new dictionary
        succeeding_activities_updated[key] = new_values
    # create an empty dictionary to store all the preceding activities of an activity
    preceding_events_dict = {}
    # use a depth-first search (DFS) algorithm to traverse the activity graph and
    # create a list of all preceding events for each activity in the dictionary for directly preceding activities
    for activity in preceding_activities:
        # empty set for all the visited activities
        visited = set()
        # list for all currently preceding events
        preceding_events = []
        dfs(preceding_activities, visited, activity, preceding_events)
        # we need to remove the last element from the list because it corresponds to the activity itself
        preceding_events_dict[activity] = preceding_events[:-1][::-1]
    # delete all possible silent transitions from preceding_events_dict (dict where all direct preceding events are stored)
    filtered_preceding_events_full = filter_silent_transitions(preceding_events_dict, silent_transitions)
    # delete all possible silent transitions from filtered_preceding_events (dict where only direct preceding events are stored)
    filtered_preceding_events = filter_silent_transitions(preceding_activities, silent_transitions)
    # delete all possible silent transitions from succeeding_activities_updated (dict where only direct preceding events are stored)
    filtered_succeeding_activities_updated = filter_silent_transitions(succeeding_activities_updated,
                                                                       silent_transitions)
    # generate a grouped df such that we can iterate through the log case by case (sort by timestamp to ensure the correct process sequence)
    grouped_df = ocel.log.log.sort_values('event_timestamp').groupby('event_execution')
    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation
    #create an empty list for all traces in the log
    trace_list = []
    for name, group in grouped_df:
        values = list(map(str, group['event_activity']))
        trace_list.append(values)
    #create an empty list for all the reversed suffixes
    suffix_lists = []
    for activities in trace_list:
        #generate the list of suffixes for the reversed traces for more efficient comparison
        reversed_activities = activities[::-1]
        suffixes = [reversed_activities[i:] for i in range(len(reversed_activities))]
        suffix_lists.append(suffixes)
    #Initialise the current trace index
    return grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions, trace_list, suffix_lists


# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [5]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [ ]:
if __name__ == '__main__':

    # generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions, trace_list, suffix_lists = negative_events_with_weighting_parallel(
        ocel, ocpn)

    current_trace_index = 0  # initialise count variable for trace
    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, trace_list, suffix_lists,
             current_trace_index, AG, DG)
            for group_key, df_group in grouped_df]

    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_with, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization, 4))

  0%|          | 0/48 [00:00<?, ?it/s]

### Happy Path Petri Net

In [6]:
happy_path__ocel = get_happy_path_log(filename)

In [7]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [8]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel, happy_path_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

100%|██████████| 48/48 [00:08<00:00,  5.87it/s]


0.3073


### Flower Model Petri Net

In [9]:
ots = ["order","item","delivery"]

In [10]:
flower_ocpn = create_flower_model(filename,ots)

In [11]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel, flower_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

100%|██████████| 48/48 [00:07<00:00,  6.10it/s]


0.965


# DS4 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [12]:
filename = "../src/data/jsonocel/DS4.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [13]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel, ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

100%|██████████| 14507/14507 [03:22<00:00, 71.46it/s] 


0.3604


### Happy Path Petri Net

In [14]:
happy_path__ocel = get_happy_path_log(filename)

In [15]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [16]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel, happy_path_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

100%|██████████| 14507/14507 [03:38<00:00, 66.45it/s] 


0.0842


### Flower Model Petri Net

In [17]:
ots = ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]

In [18]:
flower_ocpn = create_flower_model(filename,ots)

In [19]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel, flower_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(5)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

100%|██████████| 14507/14507 [03:51<00:00, 62.78it/s] 


0.6885


### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [22]:
filename_variant = "../src/data/csv/DS4_variant_log.csv" 
object_types = ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [25]:
with open("../src/data/csv/DS4_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [27]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel_variant, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(7)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

Check the arcs:   1%|          | 5242/695752 [07:50<17:13:36, 11.13it/s]


KeyboardInterrupt: 